In [55]:
import numpy as np
import pandas as pd
from numpy.linalg import inv

In [56]:
def get_data(column_names):
    train_df = pd.read_csv('./data/housing_train.txt', delim_whitespace=True, header = None)
    test_df = pd.read_csv('./data/housing_test.txt', delim_whitespace=True, header = None)
    test_df.columns = column_names
    train_df.columns = column_names
    return train_df, test_df

In [57]:
column_names = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV']
train_data, test_data = get_data(column_names)

In [58]:
train_data.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,433.000000,433.000000,433.000000,433.000000,433.000000,433.000000,433.000000,433.000000,433.000000,433.000000,433.000000,433.00000,433.000000,433.000000
mean,3.473223,11.553118,10.762125,0.066975,0.552842,6.301021,67.842032,3.800756,9.422633,398.554273,18.384758,362.30545,12.399607,22.867206
std,8.537611,23.730903,6.658463,0.250267,0.115408,0.712280,27.796861,2.055903,8.686897,167.475074,2.128271,84.56723,7.089464,9.497501
min,0.006320,0.000000,0.460000,0.000000,0.392000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.32000,1.730000,5.000000
25%,0.071510,0.000000,5.130000,0.000000,0.453000,5.888000,44.400000,2.106900,4.000000,277.000000,17.000000,376.75000,6.750000,17.200000
50%,0.229690,0.000000,8.560000,0.000000,0.532000,6.209000,74.800000,3.331700,5.000000,311.000000,18.700000,392.33000,10.580000,21.500000
75%,3.568680,12.500000,18.100000,0.000000,0.624000,6.629000,93.300000,5.214600,24.000000,666.000000,20.200000,396.90000,16.420000,25.100000
max,88.976200,100.000000,25.650000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,666.000000,21.200000,396.90000,36.980000,50.000000


In [59]:
test_data.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000
mean,4.388865,10.101351,12.562432,0.081081,0.564095,6.212865,70.805405,3.777096,10.310811,451.594595,18.866216,325.987568,13.707162,21.171622
std,8.938740,20.739906,7.012223,0.274823,0.117873,0.617129,30.071777,2.361980,8.761569,155.910311,2.326281,119.305464,6.942792,6.505351
min,0.010960,0.000000,1.760000,0.000000,0.385000,4.138000,6.800000,1.137000,1.000000,233.000000,13.000000,2.600000,2.870000,10.200000
25%,0.129418,0.000000,6.200000,0.000000,0.437000,5.875500,43.400000,2.110100,4.000000,330.000000,17.900000,321.017500,9.462500,16.750000
50%,0.548945,0.000000,12.830000,0.000000,0.584000,6.205500,84.100000,2.750250,5.000000,398.000000,20.150000,386.680000,13.100000,20.100000
75%,3.831020,0.000000,18.100000,0.000000,0.624000,6.540750,95.375000,4.653175,24.000000,666.000000,20.200000,394.620000,17.247500,24.250000
max,51.135800,80.000000,27.740000,1.000000,0.871000,8.259000,100.000000,10.585700,24.000000,711.000000,22.000000,396.900000,37.970000,42.800000


In [60]:
def normalize(dataset):
    
    maxs = dataset.max()
    mins = dataset.min()
    
    for feature in dataset.columns[:-1]:        
        for i, entry in dataset.iterrows():
            dataset.at[i, feature] = (entry[feature] - mins[feature]) / (maxs[feature] - mins[feature])
            
    return dataset

In [61]:
# train_data = normalize(train_data)
# test_data = normalize(test_data)

In [62]:
def get_weights(train_data):
    x = train_data.drop(['MEDV'], axis = 1).values
    y = train_data['MEDV'].values

    bias = np.ones(len(train_data))
    x = np.append(np.ones([len(x),1]),x,1)
 
    w = np.dot(np.dot(inv(np.dot(x.T, x)), x.T), y)
    return w

In [63]:
weights = get_weights(train_data)

In [64]:
def predict(test_data, weights):
    test_data = test_data.drop(['MEDV'], axis = 1).values
    
    bias = np.ones(len(test_data))
    test_data = np.append(np.ones([len(test_data),1]),test_data,1)
 
    preds = {}
    
    for i in range(len(test_data)):
        preds[i] = np.dot(weights, test_data[i])
        
    return preds

In [65]:
def get_mse(test_data, preds):
    test_labels = test_data['MEDV'].values
    errors = []

    for i, label in enumerate(test_labels):
        errors.append(np.square(label - preds[i]))
    
    mse = pd.Series(errors).mean()
    return mse

In [66]:
preds = predict(test_data, weights)
print('MSE for SpamBase: {}'.format(get_mse(test_data, preds)))

MSE for SpamBase: 22.638256296598747
